In [1]:
import cv2  as cv
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
from PIL import Image
import cv2

In [2]:
filename = 'sudoku.jpg'

@interact(blockSize =(0, 20,1), ksize =(1, 31,2), k=(0,0.5,0.001))
def g(blockSize=1 , ksize=1 , k=0):
    img_source = cv.cvtColor(cv.imread(filename), cv2.COLOR_BGR2RGB)
    img = img_source.copy()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv.cornerHarris(gray, blockSize ,ksize, k=k)
    dst = cv.dilate(dst, None)
    img[dst > 0.01 * dst.max()] = [255, 0, 0]
    
    plt.figure(figsize=(15,10))
    plt.subplot(121),plt.imshow(img_source,cmap = 'gray')
    plt.title('Original Image'), plt.axis('off')
    plt.subplot(122),plt.imshow(img,cmap = 'gray')
    plt.title('Detect'), plt.axis('off')


interactive(children=(IntSlider(value=1, description='blockSize', max=20), IntSlider(value=1, description='ksi…

In [3]:
from skimage import feature
from skimage import exposure
filename = 'cuaso.jpg'

@interact(orientations_num =(0, 10,1), pixels_per_cell_num =(0,52,4), bl= (0,52,4))
def g(orientations_num=9 , pixels_per_cell_num=16, bl = 2):
    img = cv.cvtColor(cv.imread(filename), cv2.COLOR_BGR2RGB)
    if len(img.shape) !=2:
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    (H, hogImage) = feature.hog(img, orientations=orientations_num, pixels_per_cell=(pixels_per_cell_num, pixels_per_cell_num),
                                    cells_per_block=(bl, bl), 
                                    visualize=True)
    print(hogImage.max())
    print(hogImage.min())
    
    hogImage =  exposure.rescale_intensity(hogImage, in_range=(0,5))
    
    print(hogImage.max())
    print(hogImage.min())
    hogImage = hogImage.astype("uint8")
    plt.figure(figsize=(15,10))
    plt.subplot(121),plt.imshow(img,cmap = 'gray')
    plt.title('Original Image'), plt.axis('off')
    plt.subplot(122),plt.imshow(hogImage,cmap = 'gray')
    plt.title('hogImage'), plt.axis('off')

interactive(children=(IntSlider(value=9, description='orientations_num', max=10), IntSlider(value=16, descript…

In [4]:
@interact(threshold1 =(0, 500,5), threshold2 =(0, 600,5), apertureSize=(0, 16,1))
def g(threshold1=0 , threshold2=0 , apertureSize=3):
    img = cv.cvtColor(cv.imread(filename), cv2.COLOR_BGR2RGB)
    assert img is not None, "file could not be read, check with os.path.exists()"
    img = cv.GaussianBlur(img,(5,5),0)
    edges = cv.Canny(img,threshold1,threshold2,apertureSize=apertureSize)
    plt.figure(figsize=(15,10))
    plt.subplot(121),plt.imshow(img,cmap = 'gray')
    plt.title('Original Image'), plt.axis('off')
    plt.subplot(122),plt.imshow(edges,cmap = 'gray')
    plt.title('Canny'), plt.axis('off')

interactive(children=(IntSlider(value=0, description='threshold1', max=500, step=5), IntSlider(value=0, descri…

In [8]:
filename = 'cuaso.jpg'
@interact(threshold1 =(0, 500,5), threshold2 =(0, 500,5), apertureSize=(1, 16,2),
        rho = (1, 50,1), theta = (0, 180,15),hreshold=(0, 500,20))
def g(threshold1=100 , threshold2=190 , apertureSize=3,rho = 1, theta = 45,threshold=200):
    img = cv.cvtColor(cv.imread(filename), cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edges = cv2.Canny(gray, threshold1, threshold2, apertureSize=apertureSize)

    # This returns an array of r and theta values
    lines = cv2.HoughLines(edges, rho=rho ,theta =  (np.pi)*theta/180, threshold=threshold)
    for r_theta in lines:
        arr = np.array(r_theta[0], dtype=np.float64)
        r, theta = arr
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*r
        y0 = b*r
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.drawMarker(img, (x1, y1), (0, 0, 255), cv2.MARKER_TILTED_CROSS, markerSize=10, thickness=1)
    print(lines.shape[0])
    plt.figure(figsize=(15,10))
    plt.subplot(121),plt.imshow(img,cmap = 'gray')
    plt.title('Original Image'), plt.axis('off')
    plt.subplot(122),plt.imshow(edges,cmap = 'gray')
    plt.title('HoughLines'), plt.axis('off')

interactive(children=(IntSlider(value=100, description='threshold1', max=500, step=5), IntSlider(value=190, de…

In [9]:
kernel1 = np.array([
    [1,0,-1],
    [1,0,-1],
    [1,0,-1]
])
filename = 'cuaso.jpg'
@interact(threshold1 =(0, 500,5), threshold2 =(0, 500,5), apertureSize=(1, 16,2),
        rho = (1, 50,1), theta = (0, 180,15),hreshold=(0, 500,20))
def g(threshold1=100 , threshold2=390 , apertureSize=3,rho = 10, theta = 45,threshold=200):
    img = cv.cvtColor(cv.imread(filename), cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray=cv.filter2D(src=gray,ddepth=-1,kernel=kernel1)
    # gray = cv2.Laplacian(gray, cv2.CV_64F)
    
    edges = cv2.Canny(gray, threshold1, threshold2, apertureSize=apertureSize)
    edges=cv.filter2D(src=edges,ddepth=-1,kernel=kernel1)
    

    # This returns an array of r and theta values
    lines = cv2.HoughLines(edges, rho=rho ,theta =  (np.pi)*theta/180, threshold=threshold)
    for r_theta in lines:
        arr = np.array(r_theta[0], dtype=np.float64)
        r, theta = arr
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*r
        y0 = b*r
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.drawMarker(img, (x1, y1), (0, 0, 255), cv2.MARKER_TILTED_CROSS, markerSize=10, thickness=1)
    print(lines.shape[0])
    plt.figure(figsize=(15,10))
    plt.subplot(121),plt.imshow(img,cmap = 'gray')
    plt.title('Original Image'), plt.axis('off')
    plt.subplot(122),plt.imshow(edges,cmap = 'gray')
    plt.title('HoughLines'), plt.axis('off')

interactive(children=(IntSlider(value=100, description='threshold1', max=500, step=5), IntSlider(value=390, de…